# 2. Preprocessing
This notebook follows our [initial data exploration](Exploration.ipynb) and refines the dataset we will be using in the project.

* [Finalize working subset](#subset)
* [Normalize](#normalize)
* [Validate](#validate)

In [1]:
import os, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from IPython.core.display import display, HTML
%matplotlib inline

In [2]:
# all available:
with open('header','r') as source:
    FEATURES = source.read().strip().split(',')

# leave 2019 out due to procedural change affecting the data?
YEARS = np.arange(2010, 2020)

<a name="subset"></a>
#### 1. Finalize subset

In [3]:
# features we are going to use (we keep original unique-key for the reference to the data source):
PICK = ('Unique Key,Agency,Complaint Type,Descriptor,Created Date,Closed Date,Address Type,Location Type,'
        'Incident Zip,Incident Address,Street Name,City,Borough,Latitude,Longitude').split(',')
INDEX = [FEATURES.index(f) for f in PICK]

In [4]:
# links to the exploration summary for reference
links = []
for i in INDEX[1:]:
    links.append('<li><a href="Data.html#2010-{}" target="_blank">{}</a></li>'.format(i, FEATURES[i]))
display(HTML('<ul>{}</ul>'.format(''.join(links))))

In [5]:
# get the filtered feature-subsets
with open('pick','w') as jobs:
    for Y in YEARS:
        jobs.write('awk -F, -v OFS=\',\' \'BEGIN{FPAT="([^,]*)|(\\"[^\\"]+\\")"}{'
                   'print $'+ ',$'.join([str(i + 1) for i in INDEX]) +'}\' '
                   './data/'+ str(Y) +'.clean.csv > ./data/'+ str(Y) +'.csv\n')
        
!parallel --jobs 10 < pick

<a name="normalize"></a>
#### 2. Normalize
In the [the exploration-notebook](Exploration.ipynb) we generated value-dictionaries for all categorical features, we can use those for the data encoding and schema-normalization. First we refine them: take the values with some threshold count to filter out the "bad" entries; and normalize some categorical fields like *Street Name*.

We can use generated dictionaries or the tokenizer function from the eploration-notebook to help us figure out how we can normalize the values in the "standard" fields like *Street Name*.

Observations: some entries are abbreviated, others not; some numbers have ordinal suffix, other do not. Below is the normalization strategy we end up with after eyeballing feature-dictionaries and tokens:

In [6]:
sub = ['STREET ST',
       'AVENUE AVE',
       'EAST E',
       'WEST W',
       'BOULEVARD BLVD',
       'PLACE PL',
       'ROAD RD',
       'PARKWAY PKWY',
       'DRIVE DR',
       'BRIDGE BR',
       'SOUTH S',
       'NORTH N',
       'HIGHWAY HWY',
       'ISLAND ISL',
       'SAINT ST',
       'LANE LN',
       'EXPY EXPWY',
       'EXPRESSWAY EXPWY']

with open('normalize','w') as job:
    job.write("#!/bin/bash\n\n")
    # remove hashes: #123
    cmd = ['sed -e \'s/\#//g\' < ./data/$1.csv']
    # abbreviate
    for s in sub:
        long, short = s.split()
        cmd.append('sed -e \'s/{}/{}/g\''.format(long, short))
    # remove ordinal suffix
    for i,s in enumerate(('TH ST ND RD' + ' TH' * 6).split()):
        cmd.append('sed -e \'s/{}{}/{}/g\''.format(i, s, i))
    # remove dots: p.o. E.
    cmd.append('sed -e \'s/\.//g\' > ./data/$1.clean.csv\n')
    job.write(' | '.join(cmd))
    
!chmod +x normalize

In [7]:
with open('jobs','w') as jobs:
    for y in YEARS:
        jobs.write('./normalize {}\n'.format(y))
    todo, n = [], len(YEARS)
    # add corresponding dictionaries
    for name in os.listdir('data'):
        if name[:5] == 'dict-' and int(name[5:-4]) in INDEX:
            jobs.write('./normalize dict-{}\n'.format(name[5:-4]))
            todo.append('<li><a href="#d{}" target="_blank">{}</a></li>'\
                            .format(name[5:-4], FEATURES[int(name[5:-4])]))
            n += 1

# 10 main files + n dictionaries
print('{} jobs total to run, then review:'.format(n))
# we might need to re-aggregate those dictionaries
display(HTML('<ul>{}</ul>'.format(''.join(todo))))

20 jobs total to run, then review:


In [8]:
# 20 jobs with 10 cores
!parallel --jobs 10 < jobs

In [9]:
# feature index and, if necessery, local version
index, local = {}, {}

Next, we look into the features from the list above one by one (only those which need the index): 
<a name="d10"></a>
##### Street Name

In [10]:
# load preprocessed summary
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('Street Name')))
# check for NaN befor setting idex (it may not be unique after processing)
data.shape[0], data.dropna().shape[0]

(37920, 7645)

In [11]:
# regroup after normalization
data = data.dropna().groupby('value').sum().reset_index()
data.shape[0]

7307

More could be done here:

In [12]:
# put w[est] e[ast] n[orth] s[outh] at the end
def rearrange(x):
    w = str(x).split()
    if len(w) > 0 and w[0] in ['W','E','S','N']:
        return ' '.join(w[1:] + [w[0]])
    return x

rearrange('E Some Street')

'Some Street E'

In [13]:
# put w[est] e[ast] n[orth] s[outh] at the end
data['value']  = data['value'].apply(rearrange)
# regroup after rearrange
data = data.groupby('value').sum()
data.shape[0]

7130

In [14]:
data = data.sort_values('total', ascending = False).astype(int)
# check less common
data.tail()

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
ROSENTHAL PL,1,3,1,2,1,2,2,2,2,1,17
NEW ENGLAND THRUWAY SERVICE RD,1,1,1,1,2,3,1,1,1,4,16
PONTON AVE,1,1,1,3,1,1,1,1,3,2,15
OLD COUNTRY RD,1,1,2,2,3,1,2,1,1,1,15
CENTREVILLE AVE,1,1,1,1,1,2,2,1,1,1,12


In [15]:
# save final for indexing
data.to_csv('./data/index-{}.csv'.format(FEATURES.index('Street Name')))

<a name="d3"></a>
##### Agency

In [16]:
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('Agency'))).set_index('value')
data.shape[0], data.dropna().shape[0]

(31, 17)

In [17]:
def highlight_nan(x):
    return 'color: crimson' if pd.isnull(x) or x == 0 else ''

data[data.isnull().any(axis = 1)].fillna(0).astype(int).style.applymap(highlight_nan)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
HRA,0,1,3819,10208,17160,22463,25059,18326,12362,4555,113953
3-1-1,1909,8158,9115,348,299,317,361,87,0,0,20594
DCAS,0,0,0,0,0,0,0,264,1168,441,1873
ACS,0,1,0,0,0,0,222,420,384,193,1220
TAX,0,0,0,0,0,0,0,52,257,152,461
DVS,0,0,0,0,0,0,0,0,0,123,123
DCP,0,0,0,0,0,0,29,24,22,14,89
DORIS,0,0,0,0,0,0,67,0,0,0,67
FDNY,0,0,0,0,0,0,0,0,60,1,61


In [18]:
# save final for indexing
data.sort_values('total', ascending = False).to_csv('./data/index-{}.csv'.format(FEATURES.index('Agency')))

<a name="d16"></a>
##### City

In [19]:
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('City')))
data.shape[0], data.dropna().shape[0]

(2489, 101)

In [20]:
data = data.groupby('value').sum().sort_values('total', ascending = False)
data.shape[0]

2459

In [21]:
# check those NaNs
data.tail(100).astype(int).style.applymap(highlight_nan)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
MELVERN,0,0,0,0,1,0,0,0,0,0,1
MELBOURNE,0,0,0,0,1,0,0,0,0,0,1
"MEDFORD NEW YORK,SUFFOLK COUNTY",0,0,0,1,0,0,0,0,0,0,1
CLEVAND,0,1,0,0,0,0,0,0,0,0,1
MEADVILLE,0,0,0,0,0,0,0,1,0,0,1
MCCARRAN,0,0,1,0,0,0,0,0,0,0,1
MCARTHUR,0,0,0,0,0,0,0,1,0,0,1
MCALLES,1,0,0,0,0,0,0,0,0,0,1
MCALLEN,1,0,0,0,0,0,0,0,0,0,1


Looks like NaN here is either misspelling or a bad entry (ANYWHERE): we drop it all.

In [22]:
data = data[data > 0].dropna()
data.astype(int)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
BROOKLYN,604021,590496,535163,540828,631071,678905,702602,755846,848926,743305,6631163
NEW YORK,365642,351366,329152,356915,412335,457738,503476,469055,498035,450914,4194628
BRONX,380242,361459,322314,325332,378414,399300,425238,435441,466654,417778,3912172
STATEN ISL,100545,99986,86908,85349,102751,108311,109933,129089,152943,125941,1101756
JAMAICA,51422,49151,43343,40961,47743,54054,54005,58853,69990,62007,531529
...,...,...,...,...,...,...,...,...,...,...,...
LAKE SUCCESS,3,3,3,2,5,1,2,3,3,2,27
FRANKLIN SQUARE,4,2,3,3,1,4,4,2,3,1,27
MT VERNON,6,1,3,2,2,1,1,2,3,3,24


In [23]:
# save final for indexing
data.to_csv('./data/index-{}.csv'.format(FEATURES.index('City')))

<a name="d25"></a>
##### Borough

In [24]:
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('Borough'))).set_index('value')
data

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
BROOKLYN,383526,563807,557259,573486,657103,704974,725597,778470,870439,772479,6587140
QUEENS,380845,424625,417416,404295,480069,531704,538563,599274,677120,605531,5059442
MANHATTAN,267492,352022,356601,387033,435849,481980,526205,491462,520179,488028,4306851
BRONX,198900,336380,341966,349021,395612,416324,441916,453024,484707,439161,3857011
STATEN ISL,89926,100424,88726,89093,105252,110549,112197,131318,154665,129208,1111358


In [25]:
# save final for indexing
data.to_csv('./data/index-{}.csv'.format(FEATURES.index('Borough')))

<a name="d7"></a>
##### Location Type 

In [26]:
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('Location Type')))
data.shape[0], data.dropna().shape[0]

(167, 95)

In [27]:
data = data.groupby('value').sum().sort_values('total', ascending = False)
data.fillna(0).astype(int).style.applymap(highlight_nan)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
value,,,,,,,,,,,
RESIDENTIAL BUILDING,671803,659438,563563,578255,613298,619614,595600,567903,609128,466974,5945576
ST/SIDEWALK,139975,142797,139402,161806,222821,305722,399386,454511,477969,497675,2942064
ST,222816,200577,198480,191197,196784,198783,209397,213324,244706,184397,2060461
RESIDENTIAL BUILDING/HOUSE,119071,114642,130929,155198,202573,219434,235618,243394,230159,225040,1876058
SIDEWALK,89531,98044,83887,85410,105175,114904,119360,193695,338666,241695,1470367
PROPERTY ADDRESS,31321,22623,19753,18524,21329,20409,18777,21460,27398,10134,211728
STORE/COMMERCIAL,6675,6021,6710,8643,17376,25889,38829,32931,31890,29944,204908
CLUB/BAR/RESTAURANT,16327,14211,15553,19880,22574,21528,22250,22116,19005,14344,187788
SENIOR ADDRESS,20779,27822,25802,17158,16624,16467,14828,13527,12865,5691,171563


In [28]:
# let's try to reduce this index (dwelling, building, 1-2, 1-,2-,3-...)
with open('location-type','w') as output:
    output.write('value,label\n"{}",\n'.format('",\n"'.join(data.index.tolist())))

# load after we done with it and saved as location-type-local
labels = pd.read_csv('location-type-local', dtype = str).set_index('value')

In [29]:
# see with our reduction
test = data.join(labels).reset_index().drop('value', axis = 1)\
        .groupby('label').sum().sort_values('total', ascending = False)

test.astype(int).style.applymap(highlight_nan)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,total
label,,,,,,,,,,,
RESIDENTIAL,858650,844176,740720,773944,858099,884256,877710,864989,889439,735094,8327077
STREET,454005,443082,423574,440609,527311,622561,732369,865577,1065585,946872,6521545
FOOD/ENTERTAINMENT,24808,23586,25797,28366,32196,32793,34633,34271,31541,25236,293227
COMMERCIAL,14963,12904,13727,15058,24700,33034,45827,42484,41977,42145,286819
PUBLIC,15508,15034,14611,14616,18650,21303,28925,27676,28058,26179,210560
SENIOR,21171,28123,26119,17457,16986,16815,15244,13910,13280,5837,174942
LOT,9043,8802,8818,9093,8514,9157,9344,9118,9411,7827,89127
MIXED USE,7900,5451,5445,4683,6039,5226,5388,8440,8884,9044,66500
RDWY/HWY,5235,5301,3021,3366,8739,7797,4733,4096,4232,3492,50012


In [30]:
# save final for indexing
data.join(labels).to_csv('./data/index-{}.csv'.format(FEATURES.index('Location Type')))
with open('./data/local-{}'.format(FEATURES.index('Location Type')), 'w') as output:
    output.write('\n'.join(test.index.tolist()))

In [31]:
# use local index for location-type
local['Location Type'] = labels.to_dict()['label']

# start from 1 (we reserve 0 for NaN)
new_index = { v:str(i + 1) for i,v in enumerate(test.index.tolist()) }
# replace with new
for value in local['Location Type']:
    key = local['Location Type'][value]
    local['Location Type'][value] = new_index[key] if key in new_index else ''

<a name="d15"></a>
##### Address Type

In [32]:
data = pd.read_csv('./data/dict-{}.clean.csv'.format(FEATURES.index('Address Type')))
data.shape[0], data.dropna().shape[0]

(5, 4)

In [ ]:
data = data.groupby('value').sum().sort_values('total', ascending = False)
data.astype(int).style.applymap(highlight_nan)

In [34]:
# save final for indexing
data.to_csv('./data/index-{}.csv'.format(FEATURES.index('Address Type')))

In [35]:
# clean pandas floating-poin nightmare
!sed -i 's/\.0,/,/g' ./data/index*
!sed -i ':a;N;$!ba;s/\.0\n/\n/g' ./data/index*

In [36]:
!ls -l data/index*

-rw-r--r-- 1 arcta arcta 341170 Feb  9 16:09 data/index-10.csv
-rw-r--r-- 1 arcta arcta    446 Feb  9 16:09 data/index-15.csv
-rw-r--r-- 1 arcta arcta   5446 Feb  9 16:09 data/index-16.csv
-rw-r--r-- 1 arcta arcta    492 Feb  9 16:09 data/index-25.csv
-rw-r--r-- 1 arcta arcta   1650 Feb  9 16:09 data/index-3.csv
-rw-r--r-- 1 arcta arcta  10354 Feb  9 16:09 data/index-7.csv


In [37]:
# make lookup dict for the index we created above
for name in os.listdir('data'):
    if name[:6] == 'index-':
        data = pd.read_csv('./data/{}'.format(name))
        feature = FEATURES[int(name[6:-4])]
        values = data['value'].tolist()
        # use local if available
        if feature in local:
            index[feature] = { x:local[feature][x] for i,x in enumerate(values) }
        else:
            index[feature] = { x:str(i+1) for i,x in enumerate(values) }

In [38]:
# use clean dict
for name in os.listdir('data'):
    if name[:5] == 'dict-' and name[-10:] == '.clean.csv':
        pd.read_csv('./data/{}'.format(name)).set_index('value').to_csv('./data/{}.csv'.format(name[:-10]))

<a name="validate"></a>
#### 3. Validate & Index
Note: this encoding meant for the storage, not for the model training.

In [39]:
# use only first numeric token, ignore the rest (apt and redundant st name)
def address(x):
    x = x or ''
    x = str(x).strip()
    if len(x) > 0:
        w = ' '.join(re.split(r'\D+', x)).strip().split()
        if len(w) > 0:
            return w[0]
    return ''

address('8768a the street')

'8768'

In [40]:
# remove non-NY or invalid zipcodes
def zipcode(x):
    try:
        x = int(str(x)[:5])
    except ValueError:
        return ''
    else:
        if 20000 > x > 10000: # NY
            return str(x)
        return ''

In [41]:
!head ./data/summary-2010-39.csv

-73.8661287701622,972
-73.92714580413967,895
-73.93211429429171,863
-73.92248337295159,755
-73.94900486982377,706
-73.9603425781195,679
-73.78309546512209,642
-73.96199943182411,641
-73.90193505991684,603
-73.886775529518,600


In [42]:
!head ./data/summary-2010-38.csv

40.87225493792927,972
40.861937381906316,895
40.85372003211803,863
40.84848255580813,755
40.6385706801265,706
40.67115570695576,679
40.64391915466994,642
40.71157025286253,641
40.81651295693585,603
40.83370844901697,600


In [43]:
# remove non-NY coords
def latitude(x):
    x = x or ''
    x = str(x).strip()
    if len(x) > 2 and x[0] == '4':
        return '{}.{}'.format(x[:2], x[2:])
    return ''

In [44]:
def longitude(x):
    x = x or ''
    x = str(x).strip()
    if len(x) > 3 and x[:2] == '-7':
        return '{}.{}'.format(x[:3], x[3:])
    return ''

Date and time transformation:

In [45]:
def to_iso(x):
    d = x.split('/')
    if len(d) < 3:
        return ''
    return '{}-{}-{}'.format(d[2],d[0],d[1])

to_iso('04/11/2018')

'2018-04-11'

In [46]:
def afternoon(x):
    t = x.split(':')
    if len(t) == 3 and int(t[0]) <= 12:
        return '{}:{}:{}'.format(int(t[0]) + 12, t[1], t[2])
    return ''

afternoon('09:45:14')

'21:45:14'

In [47]:
def daytime(x):
    t = x.split(':')
    if len(t) == 3:
        return '{:.2f}'.format((int(t[0]) + int(t[1])/60)/24)
    return ''

daytime('12:30:00')

'0.52'

In [48]:
# normalize text
def text(x):
    return (' '.join(re.sub(r'[\W\d_]+',' ', x).strip().split())).lower()

In [49]:
# put it all together
def proc(y):
    # load year data
    data = pd.read_csv('./data/{}.clean.csv'.format(y), header = None, dtype = str).astype(str)
    n,_ = data.shape
    data.columns = PICK
    data = data.set_index('Unique Key')
    
    # remove street-name from the address
    data['Incident Address'] = data['Incident Address'].apply(address)
    
    # use normalized street-name
    data['Street Name'] = data['Street Name'].apply(rearrange)
    
    # check zipcode value
    data['Incident Zip'] = data['Incident Zip'].apply(zipcode)
    
    # check lat-lng
    data['Latitude'] = data['Latitude'].apply(latitude)
    data['Longitude'] = data['Longitude'].apply(longitude)
    
    # normalize text
    data['Complaint Type'] = data['Complaint Type'].apply(text)
    data['Descriptor'] = data['Descriptor'].apply(text)
    
    # index categorical content
    for name in index:
        data[name] = data[name].apply(lambda x:index[name][x] if x in index[name] else '')
        
    # normalize date pd.to_datetime(data['Created Date'], infer_datetime_format = True)
    D = data['Created Date'].str.split(' ', expand = True)
    D.columns = ['Date','Time','AmPm']
    data['Time'] = D.apply(lambda x:x['Time'] if x['AmPm'] == 'AM' else afternoon(x['Time']), axis = 1)
    data['AmPm'] = data['Time'].apply(daytime)
    data['Created Date'] = D['Date'].apply(to_iso)
    
    # we need only date part here if available
    available = ~data['Closed Date'].isna()
    data.loc[available,'Closed Date'] = data.loc[available,'Closed Date'].apply(lambda x:to_iso(x.split()[0]))
    
    # check for NaN
    presence = ['{}: {:.2%}'.format(f, 1 - (data[f] == '').sum()/n) for f in PICK[1:]]
    print('\n\n{}:\n\t{}'.format(y,'\n'.join(presence)))
    # save final
    data.sort_values(['Created Date','AmPm']).to_csv('./data/{}.csv'.format(y))

<a name="summary"></a>

In [50]:
# check 2010 first
data = proc(2010)



2010:
	Agency: 100.00%
Complaint Type: 100.00%
Descriptor: 99.88%
Created Date: 100.00%
Closed Date: 97.61%
Address Type: 96.22%
Location Type: 70.18%
Incident Zip: 93.12%
Incident Address: 73.09%
Street Name: 74.79%
City: 93.18%
Borough: 65.00%
Latitude: 88.23%
Longitude: 88.23%


In [ ]:
!head data/2010.csv

In [52]:
# save final labels
!head -n1 ./data/2010.csv > features-local

In [53]:
# get the rest done
for y in YEARS[1:]: proc(y)



2011:
	Agency: 100.00%
Complaint Type: 100.00%
Descriptor: 99.89%
Created Date: 100.00%
Closed Date: 93.25%
Address Type: 95.58%
Location Type: 71.42%
Incident Zip: 92.72%
Incident Address: 73.89%
Street Name: 75.52%
City: 92.79%
Borough: 90.60%
Latitude: 87.67%
Longitude: 87.67%


2012:
	Agency: 100.00%
Complaint Type: 100.00%
Descriptor: 99.88%
Created Date: 100.00%
Closed Date: 98.29%
Address Type: 95.85%
Location Type: 70.93%
Incident Zip: 92.90%
Incident Address: 75.25%
Street Name: 76.71%
City: 92.95%
Borough: 98.10%
Latitude: 89.01%
Longitude: 89.00%


2013:
	Agency: 100.00%
Complaint Type: 100.00%
Descriptor: 99.86%
Created Date: 100.00%
Closed Date: 98.10%
Address Type: 94.06%
Location Type: 71.71%
Incident Zip: 91.60%
Incident Address: 76.69%
Street Name: 78.24%
City: 91.64%
Borough: 97.99%
Latitude: 88.33%
Longitude: 88.33%


2014:
	Agency: 100.00%
Complaint Type: 100.00%
Descriptor: 99.89%
Created Date: 100.00%
Closed Date: 98.20%
Address Type: 95.81%
Location Type: 71.64

Observation: the missing data trend shows consistent decline.

In [54]:
!rm ./data/*.clean.csv
!rm ./data/summary-20*

The textual features *Complaint Type, Descriptor* are processed in [the topic-modeling notebook](Taxonomy.ipynb). At this point we are ready to import preprocessed csv into the database.